In [ ]:
"""
4/16

Purpose: implement the juke box project

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from os import sys

library_path = Path("/meshAfterParty/meshAfterParty/")
library_path_str = str(library_path.absolute())

if library_path_str not in sys.path:
    sys.path.append(library_path_str)

import numpy_utils as nu
import numpy as np
import polynomial_utils as pu
import dsp_utils as dspu
import matplotlib.pyplot as plt
from numpy import sin,cos,pi,exp
import time

# Loading the juke data

In [3]:
import scipy.io as sio
import filter_banks_utils as fu

In [4]:
music_dir = Path("./Music_Data/")
list(music_dir.iterdir())

[WindowsPath('Music_Data/Mike_Juke_IntData.mat'),
 WindowsPath('Music_Data/TrackAA.mat'),
 WindowsPath('Music_Data/TrackBB.mat'),
 WindowsPath('Music_Data/TrackCC.mat'),
 WindowsPath('Music_Data/TrackDD.mat'),
 WindowsPath('Music_Data/TrackEE.mat')]

In [5]:
dtype_juke = "double"
juke_data = (sio.loadmat('Music_Data/Mike_Juke_IntData.mat')["mike_jukeI"].astype(dtype_juke)/6400).flatten()

In [6]:
L = 32

# Loading the track data

In [7]:
track_dir = [k for k in list(music_dir.iterdir()) if "Track" in str(k)]
track_data = dict()
for curr_file in track_dir: 
    curr_mat = sio.loadmat(curr_file)
    file_name = curr_file.stem
    curr_key = file_name[0].lower() + file_name[1:]
    curr_mat = curr_mat[curr_key]
    track_data[curr_key] = curr_mat.flatten()

In [8]:
# import filter_banks_utils as fu
# fu.play_track(audio = track_data["trackEE"].copy(),
# sampling_rate = 44100)

# Trying to demodulate the individual tracks from the main data

In [10]:
from scipy import signal
import dsp_utils as dspu

In [11]:
verbose = True
plot_fir_Hdw = False

In [12]:
filter_length=511
delta = 0.008
bandwidth = pi/32

passband_freq=bandwidth - delta
stopband_freq=bandwidth + delta
sampling_freq=2*pi

bands = [0,passband_freq,stopband_freq,pi]
desired = [1,1,0,0]

hhh = signal.firls(filter_length, bands, desired, fs=sampling_freq)
if verbose:
    print(f"Lenght of filter = {len(hhh)}")
    m_idx = int((len(hhh)+1)/2 - 1)
    print(f"Proof Even Symmetric: np.linalg.norm(hhh[:{m_idx+1}] - np.flip(hhh[{m_idx}:])) = {np.linalg.norm(hhh[:m_idx+1] - np.flip(hhh[m_idx:]))}")

if plot_fir_Hdw:
    sdspu.sequence_to_plot_Hdw_magnitude(hhh)

Lenght of filter = 511
Proof Even Symmetric: np.linalg.norm(hhh[:256] - np.flip(hhh[255:])) = 0.0


# wanting to extract a song from the raw data

In [ ]:
"""
trackAA: classic violin (k = 4, cosine)
trackBB: guitar (with drums): ground control to major Tom (k = 7, sin)
trackCC: guitar: you're everlasting summer (are you reeling in the years) (k = 9: sin)
trackDD: last thing I remember I was running for the door (hotel california) (k = 7 cosine)
trackEE: coldplay: all the signs , some get made and some get (speed of sound) (k = 13, cosine)


Psuedocode: 
1) Create an sin or cosine sequence that applies the 
certain modulation you want to apply
2) multiply the sinusoid with the raw data
 -- or can do exponential and multiply by 2

3) convolve the new sequence with the filter 
(to pass it through lowpass) filter
4) downsample by 32 starting at some start index
5) Play the song
"""

# loading the matlab filter to start out

In [ ]:
#hhh_mat = sio.loadmat("firls")["hhh"].flatten()

In [ ]:
ex_conv = np.convolve(juke_data,hhh,"same")
ex_conv

In [ ]:
ex_conv[0]

In [ ]:
np.sum(np.hstack([np.zeros(255),juke_data[:256]])*hhh)

In [ ]:
index = 1001
np.sum(juke_data[index-255:index+256]*hhh)

In [ ]:
track_1,track_2 =fu.extract_track_from_juke_data_by_mod_low_pass(juke_data,
                                                    filt=hhh,
                                                    #filt=hhh_mat,
                                                    kkk=kkk,
                                                    downsample_shift = -1,
                                                    verbose=False)
current_rms = fu.RMS(track_1,track_data["trackAA"])

In [ ]:
kkk = 13
rms_values = [] 
#downsample_shifts = np.arange(32)
downsample_shifts = [0]
for d in downsample_shifts:
    track_1,track_2 = fu.extract_track_from_juke_data_by_mod_low_pass(juke_data,
                                                    filt=hhh,
                                                    #filt=hhh_mat,
                                                    kkk=kkk,
                                                    downsample_shift = d,
                                                    verbose=False)
    current_rms = fu.RMS(track_1,track_data["trackEE"])
    #rms_values.append(current_rms)
    print(f"For downsample shift {d}: rms = {current_rms}")

# Coding up the filter bank

In [ ]:
"""
1) Generate the polyphase filters of your filter
- with a zero at the front or the end
2) Rearrange the input data into an Lx(N/L) block
3) convolve each row with the required polyphase
4) Pass each column through an FFT to get a sequence of outputs
--> collect each output as column and put in matrix (output)
"""

In [ ]:
verbose = True

In [ ]:
#1) Making the polyphase filters
import numpy_utils as nu
nu.turn_off_scientific_notation()

st = time.time()
p_filters = fu.filter_to_polyphase_filters(hhh,
                                        pad_mode = "start",
                                       verbose=True)
p_filters_flip = np.flip(p_filters,axis=0)
p_filters_flip[:,0]



#2) Rearrange the input data into an Lx(N/L) block
juke_data_banks = juke_data.reshape(-1,L).T
print(f"juke_data_banks.shape = {juke_data_banks.shape}")

if verbose:
    print(f"Total time for polyphase generation and data reshape= {time.time() - st}")

In [ ]:
#3) convolve each row with the required polyphase
st = time.time()
convolve_results = np.array([np.convolve(jd,pf,"same") for jd,pf in zip(juke_data_banks,p_filters_flip)]).T
if verbose:
    print(f"convolve_results.shape = {convolve_results.shape}")
    print(f"Total time for polyphase convolution= {time.time() - st}")

In [ ]:
#4) Pass each column through an FFT to get a sequence of outputs
#--> collect each output as column and put in matrix (output)

#Source: https://stackoverflow.com/questions/54535743/numpy-fft-numpy-fft2-and-fftw-for-2d-arrays
st = time.time()
filter_bank_output = 2*np.fft.fft(convolve_results)

if verbose:
    print(f"filter_bank_output.shape = {filter_bank_output.shape}")
    print(f"Total time for fft of filterbank= {time.time() - st}")

In [ ]:
# testing the output of the filter bank
fu.play_track(audio = np.real(filter_bank_output[:,4]),
sampling_rate = 44100)


In [ ]:
fu.RMS(np.real(filter_bank_output[:,4]),track_data["trackAA"])

# Part C

In [13]:
#n_pads = int((np.floor(len(juke_data)/len(hhh)) + 1)*(len(hhh))-len(juke_data))
n_pads = len(hhh) - L
juke_data_len = len(juke_data)
# juke_data_pad = np.hstack([,
#                            juke_data,np.floor(np.zeros(n_pads)/2).astype("int"),
#                           ])
juke_data_pad = np.hstack([np.zeros(255),
                          juke_data,np.zeros(255 - 31)])

In [15]:
mod_info = dict(AA=dict(func=np.cos,k = 4),
                      BB=dict(func=np.sin,k = 7),
                      CC=dict(func=np.sin,k = 9),
                       DD=dict(func=np.cos,k = 7),
                      EE=dict(func=np.cos,k = 13))

In [ ]:
"""
Pseudocode: To create a final DtD matrix and DtR that will
hold the values for the least square solution

Psuedocode: 
1) Create an empty DtD and DtR
2) Iterate through all of the tracks we have the info for:

a: get the actual track
b. modulate the current D according to the track info
c. create a current DtD out of the modulated D (add to master)
d. create a current DtR from the modulated D and the actual track (add to master)

3) invert the DtD
4) matrix multiply DtD inverse and DtR

"""

DTD = np.zeros((len(hhh),len(hhh)))
DTR = np.zeros((len(hhh),1))

for track_suffix in mod_info.keys():
    print(f"Working on track_suffix = {track_suffix}")
    #a: get the actual track
    curr_track_data = track_data[f"track{track_suffix}"]

    #b. modulate the current D according to the track info
    mod_func = mod_info[track_suffix]["func"]
    mod_k = mod_info[track_suffix]["k"]
    
    DTD_curr = fu.modulated_D(juke_data,
                  filter_len = len(hhh),
                  L = L,
                  mod_func = mod_func,
                  mod_k = mod_k)
    DTD += np.matmul(DTD_curr.T,DTD_curr)
    
    DTR_curr += np.matmul(DTD_curr.T,curr_track_data.reshape(len(curr_track_data),1))
    break
    

In [ ]:
4*pi/16

In [30]:
fu.RMS(curr_audio.flatten(),track_data["trackAA"])

0.0012918325162346092

In [ ]:
#curr_track = "trackEE"

D_total = np.zeros((int(len(juke_data)/L),len(hhh)),dtype="float16")
for idx in range(int(len(juke_data)/L)):
    for curr_track in mod_info.keys():
        print()
        D_total[idx,:] = juke_data_pad[L*idx:L*idx+len(hhh)]
        st_shift = int((len(hhh)-1)/2)
        cos_seq = mod_info[curr_track]["func"](2*pi/L*mod_info[curr_track]["k"]*np.arange(-st_shift,len(hhh)-st_shift)).astype("float16")*2
        D_mod = (D_total*cos_seq)
#     for k in mod_info.keys():
#         curr_k = mod_info[k]["k"]
#         curr_func = mod_info[k]["func"]
#         mod_info[k]["mod_term"][idx,:] = curr_func(2*pi/L*curr_k*np.arange(L*idx,L*idx+len(hhh)))

In [28]:
DTD_curr[499,99]

-0.439

In [ ]:
"""
Least Squares Finite Element Method 
"""

In [ ]:
"""
trackAA: classic violin (k = 4, cosine)
trackBB: guitar (with drums): ground control to major Tom (k = 7, sin)
trackCC: guitar: you're everlasting summer (are you reeling in the years) (k = 9: sin)
trackDD: last thing I remember I was running for the door (hotel california) (k = 7 cosine)
trackEE: coldplay: all the signs , some get made and some get (speed of sound) (k = 13, cosine)

"""

In [ ]:
D_total

In [ ]:
fu.play_track(audio = track_1,
sampling_rate = 44100)

In [ ]:
if track_2 is not None:
    fu.play_track(audio = track_2,
    sampling_rate = 44100)
else:
    print("Track 2 was none")

In [ ]:


"""
trackAA: classic violin (k = 4, cosine)
trackBB: guitar (with drums): ground control to major Tom (k = 7, sin)
trackCC: guitar: you're everlasting summer (are you reeling in the years) (k = 9: sin)
trackDD: last thing I remember I was running for the door (hotel california) (k = 7 cosine)
trackEE: coldplay: all the signs , some get made and some get (speed of sound) (k = 13, cosine)


k = 0: cos: octopus garder sin: None
k = 1: cos: folk (finding my way home)  sin: frank sinatra (just the way you look tonight)
k = 2: cos: the who (wasted) sin: behtoven
k = 3: cos: adele sin: jackson brown
k = 4: cos: trackAA  sin: ?? (drove in my car)
k = 5: cos: (slow wangy guitar) sin: pink floyd (darkside of the moon)
k = 6: cos: (the band?) sin: eric clapton
k = 7: cos: trackDD sin: trackBB (david bowie)
k = 8: cos: green slaves sin: some girl singer
k = 9: cos: no eye deer sin: trackCC
k = 10: 
k = 11: cos pink floyd sin: some guitar 
k = 12: cos: strained rock (eric clapton)   sin: who ()
k = 13: cos: trackEE (cold play)


"""

In [ ]:
# Experiment for 

In [ ]:
n = np.arange(0,10*pi,1/(2*pi))
sin_w = np.sin(0.3*pi*n)
new_wave = sin_w*sin(2*n)
plt.plot(n,sin_w)
plt.plot(n,new_wave)


In [ ]:
fir_firls = signal.firls(filter_length, [, desired, fs=sampling_freq)